In [19]:
import numpy as np # calculations with arrays
import pandas as pd # user-friendly DataFrames for data representation
import sklearn # machine learning algorithms
from sklearn import ensemble, linear_model
from sklearn.metrics import log_loss
from sklearn import cross_validation
from sklearn import grid_search
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer as DV
import xgboost as xgboost
import matplotlib.pyplot as plt # import plot functions
# necessary to plot in jupyter notebook:
%matplotlib inline
import seaborn as sns # make plots beautiful
from scipy import sparse
from itertools import product
from sklearn.feature_extraction import FeatureHasher

In [4]:
transactions = pd.read_csv('transactions.csv')
customers_gender = pd.read_csv('customers_gender_train.csv')

In [5]:
cuses_test = list(set(transactions.customer_id.unique().tolist()).difference(customers_gender.customer_id.unique()))
all_cuses = transactions.customer_id.unique()
all_mcc = transactions.mcc_code.unique()

In [6]:
transactions = transactions[transactions.amount < 0].copy()
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [7]:
transactions.day += 29 - transactions['day'].max()%30

In [8]:
transactions['month_num'] = (transactions.day) // 30
transactions['year_num'] = (transactions.day) // 365


In [9]:
test_transactions = transactions[transactions.month_num == 15]
train_transactions = transactions[transactions.month_num < 15]

In [10]:
test_transactions = test_transactions.set_index('customer_id')
test_transactions = test_transactions.loc[cuses_test]
test_transactions = test_transactions.reset_index()

In [11]:
grid = list(product(*[all_cuses, all_mcc, range(10, 15)]))
train_grid = pd.DataFrame(grid, columns = ['customer_id', 'mcc_code', 'month_num'])

In [12]:
test_grid = list(product(*[cuses_test, all_mcc]))       
test_grid = pd.DataFrame(test_grid, columns = ['customer_id', 'mcc_code'])
test_grid['month_num'] = 15

In [13]:
test = pd.merge(test_grid,
         test_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

In [15]:
train = pd.merge(train_grid,
         train_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

In [17]:
for month_shift in range(1, 6):
    train_shift = train.copy()
    train_shift['month_num'] = train_shift['month_num'] + month_shift
    train_shift = train_shift.rename(columns={"amount" : 'amount_{0}'.format(month_shift)})  
    train_shift = train_shift[['year_num', 'month_num', 'customer_id', 'mcc_code', 'amount_{0}'.format(month_shift)]]

    train = pd.merge(train, train_shift, 
                                  on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)
    test = pd.merge(test, train_shift, 
                                 on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)

In [20]:
hasher = FeatureHasher(n_features=10000, input_type='string')
train_sparse = \
    hasher.fit_transform(train[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())

In [ ]:
test_sparse = \
    hasher.transform(test[['year_num', 'month_num', 'customer_id', 'mcc_code']].astype(str).as_matrix())